<a href="https://colab.research.google.com/github/BowmanFox/fast-stable-diffusion/blob/main/dreambooth_runpod_joepenna_dogarrow_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Furry* Dreambooth
### Notebook implementation by Joe Penna (@MysteryGuitarM on Twitter) - Improvements by David Bielejeski
### Furry specific modifications by dogarrowtype#7324
### Join us at: https://discord.gg/furrydiffusion

Dreambooth is a technique that runs on top of StableDiffusion. It teaches the model the details of your specific character, and if done right, can reproduce them with high accuracy.


## This notebook requires a GPU with 24GB of VRAM. A 3090 or an A5000 on runpod will do the job.
Pick a runpod with a fast upload speed, to download the model at the end faster. Choose the standard pytorch template.

### Instructions
- Run each code cell, follow the instructions, but choose whether you want to use regularization images.

Modified from:
https://github.com/JoePenna/Dreambooth-Stable-Diffusion

This notebook has been designed and tested to work on https://runpod.io

Be careful, the predefined model is known to produce nsfw content. DO NOT use if you are below the legal age of majority in your country.

## Build Environment

In [1]:
#!git clone https://github.com/JoePenna/Dreambooth-Stable-Diffusion
%cd Dreambooth-Stable-Diffusion
!git checkout 605faeaa2f0656bf31ab79749fb5d96b3ca2e69f
!git pull

C:\Users\Niko\Dreambooth-Stable-Diffusion


HEAD is now at 605faea Merge pull request #59 from msoucy/patch-1
You are not currently on a branch.
Please specify which branch you want to merge with.
See git-pull(1) for details.

    git pull <remote> <branch>



In [3]:
# BUILD ENV
!pip install omegaconf
!pip install einops
!pip install pytorch-lightning==1.6.5
!pip install test-tube
!pip install transformers
!pip install kornia
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install setuptools==59.5.0
!pip install pillow==9.0.1
!pip install torchmetrics==0.6.0
!pip install -e .
!pip install protobuf==3.20.1
#!pip install gdown
!pip install -qq diffusers["training"]==0.3.0 transformers ftfy
!pip install -qq "ipywidgets>=7,<8"
!pip install huggingface_hub
!pip install ipywidgets==7.7.1
!pip install captionizer==1.0.1

Obtaining taming-transformers from git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
  Cloning https://github.com/CompVis/taming-transformers.git (to revision master) to c:\users\niko\dreambooth-stable-diffusion\src\taming-transformers
  Resolved https://github.com/CompVis/taming-transformers.git to commit 3ba01b241669f5ade541ce990f7650a3b8f65318
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py develop for taming-transformers


  Running command git clone --filter=blob:none --quiet https://github.com/CompVis/taming-transformers.git 'C:\Users\Niko\Dreambooth-Stable-Diffusion\src\taming-transformers'


Obtaining clip from git+https://github.com/openai/CLIP.git@main#egg=clip
  Cloning https://github.com/openai/CLIP.git (to revision main) to c:\users\niko\dreambooth-stable-diffusion\src\clip
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py develop for clip


  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\Niko\Dreambooth-Stable-Diffusion\src\clip'


Obtaining file:///C:/Users/Niko/Dreambooth-Stable-Diffusion
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py develop for latent-diffusion


## Download the model to train on top of
Swap this link out here, if you want to use something other than ye18.

In [5]:
#!wget https://sexy.canine.wf/file/yiffy-ckpt/yiffy-e18.ckpt -O /workspace/Dreambooth-Stable-Diffusion/model.ckpt

# Regularization Images (No longer recommended)
## Skip this part unless you know what it does.
### Expand if you really want to.

This is NOT the part that contains the character you want to train on. It is a separate step that helps guide the model toward what you want, but has little to do with your desired character.

For furry content, ~~regularization images help. They (seem) to prime the model to focus on the area you'd like to change.~~

## Regularization images seem to cause more harm than help for furry content. I no longer advise using them.

### But you can still use them if you want. Just expand this block.

Change "self_generated_files_prompt" to be closest to what you want. 

**For instance: change species, gender, color.**

Try to keep it close to the example and simple. Don't go heavy on the detail.

The prompt system here is not as advanced as Automatic1111, so don't do anything fancy.

In [2]:
# GENERATE 200 images - Optional
uses_reg = True
self_generated_files_prompt = "male grey_fur canid"
self_generated_files_count = 512

!python scripts/stable_txt2img.py \
 --seed 69 \
 --ddim_eta 0.0 \
 --n_samples 1 \
 --n_iter "{self_generated_files_count}" \
 --scale 10.0 \
 --ddim_steps 50 \
 --ckpt model.ckpt \
 --prompt "{self_generated_files_prompt}"


Loading model from model.ckpt
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64,

Global seed set to 69
Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.11.mlp.fc1.weight', 'vision_model.encoder.layers.17.mlp.fc1.bias', 'vision_model.encoder.layers.12.self_attn.out_proj.weight', 'vision_model.encoder.layers.10.self_attn.q_proj.weight', 'vision_model.encoder.layers.18.mlp.fc2.bias', 'vision_model.encoder.layers.17.mlp.fc1.weight', 'vision_model.encoder.layers.13.mlp.fc2.weight', 'vision_model.encoder.layers.18.self_attn.q_proj.weight', 'vision_model.encoder.layers.8.layer_norm2.weight', 'vision_model.encoder.layers.13.layer_norm1.bias', 'vision_model.encoder.layers.13.mlp.fc2.bias', 'vision_model.encoder.layers.6.self_attn.k_proj.bias', 'vision_model.encoder.layers.3.layer_norm1.bias', 'vision_model.encoder.layers.9.self_attn.v_proj.bias', 'vision_model.encoder.layers.1.mlp.fc1.bias', 'vision_model.encoder.layers.23.self_attn.v_proj.bias', 'vision_model.encoder.layers.4.

In [25]:
dataset="CUM"
!mkdir regularization_images\CUM
!copy outputs\txt2img-samples\samples\*.* regularization_images\CUM

A subdirectory or file regularization_images\CUM already exists.


 Volume in drive C is Windows 11
 Volume Serial Number is 7692-858C

 Directory of C:\Users\Niko\Dreambooth-Stable-Diffusion

05/26/2023  07:30 AM    <DIR>          .
05/26/2023  07:29 AM    <DIR>          ..
05/26/2023  06:16 AM    <DIR>          -p
05/26/2023  05:40 AM                81 .gitignore
05/26/2023  05:04 AM    <DIR>          assets
05/26/2023  05:05 AM    <DIR>          bowman
05/26/2023  05:04 AM    <DIR>          configs
05/26/2023  05:04 AM    <DIR>          data
05/26/2023  05:40 AM             8,671 dreambooth_colab_joepenna.ipynb
05/26/2023  05:40 AM            14,709 dreambooth_runpod_joepenna.ipynb
05/26/2023  05:40 AM               770 environment.yaml
05/26/2023  05:04 AM    <DIR>          evaluation
05/26/2023  05:04 AM         7,178,600 gdrive
05/26/2023  05:41 AM    <DIR>          latent_diffusion.egg-info
05/26/2023  05:40 AM    <DIR>          ldm
05/26/2023  05:04 AM             1,146 LICENSE
05/26/2023  07:24 AM    <DIR>          logs
05/26/2023  05:40 AM  

In [4]:
# Zip up the files for downloading and reuse.
# Download this file locally so you can reuse during another training on this dataset
#!apt update
#!apt install -y zip
!powershell Compress-Archive regularization_images/{dataset} regularization_images.zip

Compress-Archive : The path 'regularization_images/CUM' either does not exist or is not a valid file system path.
At line:1 char:1
+ Compress-Archive regularization_images/CUM regularization_images.zip
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidArgument: (regularization_images/CUM:String) [Compress-Archive], InvalidOperation 
   Exception
    + FullyQualifiedErrorId : ArchiveCmdletPathNotFound,Compress-Archive
 


## OR Upload regularization images generated separately to
`/workspace/Dreambooth-Stable-Diffusion/regularization_images/dataset1`

Don't do both.

In [5]:
uses_reg = True
dataset="dataset1"
!mkdir -p regularization_images/{dataset}

The syntax of the command is incorrect.


Generate them on your own pc, gather them, and upload to the folder. If you *really* want to skip regularization, delete `--reg_data_root "{reg_data_root}"` from the training block.

# Training data
# Upload your training images

In [6]:
!mkdir training_images

A subdirectory or file training_images already exists.


Upload 100-200 images of your character to

```
/workspace/Dreambooth-Stable-Diffusion/training_images
```

Be sure to upload an *even* amount of images, otherwise the training inexplicably stops at 1500 steps.

## The images should be:

- Solos only!
- Square.
- 512x512.
- Cropped to contain the most recognizable part of your character.
- Don't worry about legs/arms being cropped off, the most important part is the head. The model will make up the rest.
- A variety of angles and shots. The model can become overfitted if you only do frontal shots, for instance.
- If your character has very distinctive features or markings you want to preserve, make sure they show up in the data often.
- Variations of the same image (alt versions/edits) are fine, but don't do too many of the same.
- DON'T do black and white or uncolored sketches.
- Get as close as possible to the output you'd like to see.

A good idea is to use imagemagick for automated cropping. No, it's not the fancy kind that uses algorithms to focus on faces, but it totally works.

You're mostly on your own for cropping, but here's some...
## Hints for cropping commands:
Windows:

`magick *.* -gravity North -crop 1:1 -resize 512x512 -format png ..\raw1\image-%d.png`

Linux:

`find -maxdepth 1 -type f -execdir magick '{}' -gravity North -crop 1:1 -resize 512x512 ../ready1/'{}.png' \;`

https://imagemagick.org/script/download.php


## Use BIRME, it's by far the best option for bulk crop + resize:
https://www.birme.net/?target_width=512&target_height=512&rename=ORIGINAL-NAME_&border_width=-1

# Training

## This is where the magic happens.

Some key things to change:
- `"project_name"`: Change project name to whatever you want. Avoid spaces for everything.
- `"max_training_steps"`: 2000-4000 is a good range. Choose around 2000 if you have 100 images, and around 4000 if you have 200 images.
- `"class_word"`: Keep as anthro. Or feral. Or... whatever kind of thing you're doing. But it should be extremely simple, and something the model already recognizes easily. Only matters if using regularization images.
- `"token"`: Pick a longer, made-up word the model has never seen before. Can really be anything, but make it something like "blaiddwolfyfromeldenring". "Charname+universe" is easier to remember, imo. This is what you will type in to activate it, so... don't make it insane.

Once you've got those set, you're ready to train. It should take about an hour on a 3090 for 2500 steps.

Do not worry. It will stop at exactly the number of steps you set, even if the progress bar there looks otherwise. The estimate is usually wildly wrong. There is a separate internal counter that is not shown.

In [7]:
# Training

# This is just the name for the training run, not the token.
project_name = "bowman"

# MAX STEPS
# How many steps do you want to train for?
max_training_steps = 13000

# Match class_word to the category of the regularization images you chose above.
class_word = "anthro" # Anthro or feral are the best choices.

# This is the unique token you are teaching the stable diffusion model.
token = "ween" #Can be anything, but pick something rare and easy to remember/type.

dataset="CUM"
img_data_root = "bowman"
#reg_data_root = "regularization_images/" + dataset
#     --reg_data_root "{reg_data_root}" \

!python "main.py" \
    --base configs/stable-diffusion/v1-finetune.yaml \
    -t \
    --actual_resume "model.ckpt" \
    -n "{project_name}" \
    --gpus 0, \
    --data_root "{img_data_root}" \
    --max_training_steps {max_training_steps} \
    --class_word "{class_word}" \
    --token "{token}" \
    --no-test 


Running on GPUs 1,
Loading model from model.ckpt
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 64, 64) = 16384 dimensions.
making attention of type 'vanilla' with 512 in_channels
Restored from model.ckpt with 12 missing and 0 unexpected keys
Missing Keys: ['betas', 'alphas_cumprod', 'alphas_cumprod_prev', 'sqrt_alphas_cumprod', 'sqrt_one_minus_alphas_cumprod', 'log_one_minus_alphas_cumprod', 'sqrt_recip_alphas_cumprod', 'sqrt_recipm1_alphas_cumprod', 'posterior_variance', 'posterior_log_variance_clipped', 'posterior_mean_coef1', 'posterior_mean_coef2']
Monitoring val/loss_simple_ema as checkpoint metric.
Merged modelckpt-cfg: 
{'target': 'pytorch_lightning.callbacks.ModelCheckpoint', 'params': {'dirpath': 'logs\\bowman2023-05-26T07-24-10_bowman\\checkpoints', 'filename': '{epoch:06}', 'verbose': True, 'save_last': True, 'monitor': 'val/loss_simple_ema', 'save_t

Global seed set to 23
Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.12.self_attn.out_proj.weight', 'vision_model.encoder.layers.11.self_attn.k_proj.bias', 'vision_model.encoder.layers.8.layer_norm1.weight', 'vision_model.encoder.layers.15.layer_norm1.bias', 'vision_model.encoder.layers.22.self_attn.q_proj.weight', 'vision_model.encoder.layers.3.mlp.fc2.bias', 'vision_model.encoder.layers.22.mlp.fc1.weight', 'vision_model.encoder.layers.7.self_attn.v_proj.weight', 'vision_model.encoder.layers.21.layer_norm1.weight', 'vision_model.encoder.layers.22.mlp.fc2.bias', 'logit_scale', 'vision_model.encoder.layers.13.layer_norm1.bias', 'vision_model.encoder.layers.8.layer_norm2.bias', 'vision_model.encoder.layers.5.layer_norm1.weight', 'vision_model.encoder.layers.15.self_attn.v_proj.weight', 'vision_model.encoder.layers.18.self_attn.q_proj.bias', 'vision_model.encoder.layers.12.self_attn.k_proj.

In [8]:
# Training

# This is just the name for the training run, not the token.
project_name = "Death"

# MAX STEPS
# How many steps do you want to train for?
max_training_steps = 60000

# Match class_word to the category of the regularization images you chose above.
class_word = "anthro" # Anthro or feral are the best choices.

# This is the unique token you are teaching the stable diffusion model.
token = "ImGayStraightUp" #Can be anything, but pick something rare and easy to remember/type.


DATASET="CUM"
img_data_root = "training_images"
#reg_data_root = "regularization_images/" + dataset
#    --reg_data_root "{reg_data_root}" \

!python "main.py" \
    --base configs/stable-diffusion/v1-finetune_unfrozen.yaml \
    -t \
    --actual_resume "model.ckpt" \
    -n "{project_name}" \
    --gpus 1, \
    --data_root "{img_data_root}" \
    --max_training_steps {max_training_steps} \
    --class_word "{class_word}" \
    --token "{token}" \
    --no-test 


Running on GPUs 1,
Loading model from model.ckpt
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 64, 64) = 16384 dimensions.
making attention of type 'vanilla' with 512 in_channels
Restored from model.ckpt with 12 missing and 0 unexpected keys
Missing Keys: ['betas', 'alphas_cumprod', 'alphas_cumprod_prev', 'sqrt_alphas_cumprod', 'sqrt_one_minus_alphas_cumprod', 'log_one_minus_alphas_cumprod', 'sqrt_recip_alphas_cumprod', 'sqrt_recipm1_alphas_cumprod', 'posterior_variance', 'posterior_log_variance_clipped', 'posterior_mean_coef1', 'posterior_mean_coef2']
Monitoring val/loss_simple_ema as checkpoint metric.
Merged modelckpt-cfg: 
{'target': 'pytorch_lightning.callbacks.ModelCheckpoint', 'params': {'dirpath': 'logs\\training_images2023-05-26T07-24-37_Death\\checkpoints', 'filename': '{epoch:06}', 'verbose': True, 'save_last': True, 'monitor': 'val/loss_simple_ema',

Global seed set to 23
Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.22.layer_norm2.weight', 'vision_model.encoder.layers.16.self_attn.k_proj.weight', 'vision_model.encoder.layers.5.self_attn.q_proj.bias', 'vision_model.encoder.layers.1.mlp.fc2.bias', 'vision_model.encoder.layers.7.mlp.fc2.bias', 'vision_model.encoder.layers.0.layer_norm2.bias', 'vision_model.encoder.layers.9.self_attn.out_proj.bias', 'vision_model.encoder.layers.21.self_attn.k_proj.weight', 'vision_model.encoder.layers.13.mlp.fc2.bias', 'vision_model.encoder.layers.19.mlp.fc1.bias', 'vision_model.encoder.layers.17.self_attn.k_proj.bias', 'vision_model.encoder.layers.1.layer_norm2.bias', 'vision_model.encoder.layers.4.layer_norm2.bias', 'vision_model.encoder.layers.16.self_attn.q_proj.weight', 'vision_model.encoder.layers.3.mlp.fc1.weight', 'vision_model.encoder.layers.9.layer_norm2.bias', 'vision_model.encoder.layers.8.

## Copy and name the checkpoint file

Also prints the sha256sum, very helpful for verification/sharing your dreambooth model publicly. Take note of it.

In [9]:
# Copy the checkpoint into the `trained_models` folder

directory_paths = !ls -d logs/*
last_checkpoint_file = directory_paths[-1] + "/checkpoints/last.ckpt"
training_images = !find training_images/*
date_string = !date +"%Y-%m-%dT%H-%M-%S"
file_name = date_string[-1] + "_" + project_name + "_" + str(len(training_images)) + "_training_images_" +  str(max_training_steps) + "_max_training_steps_" + token + "_token_" + class_word + "_class_word.ckpt"

file_name = file_name.replace(" ", "_")

!mkdir -p trained_models
!mv "{last_checkpoint_file}" "trained_models/{file_name}"
!sha256sum "trained_models/{file_name}"

print("Download your trained model file from trained_models/" + file_name + " and use in your favorite Stable Diffusion repo!")

A subdirectory or file -p already exists.
Error occurred while processing: -p.
A subdirectory or file trained_models already exists.
Error occurred while processing: trained_models.
'mv' is not recognized as an internal or external command,
operable program or batch file.


Download your trained model file from trained_models/Enter_the_new_date:_(mm-dd-yy)__Death_1_training_images_60000_max_training_steps_ImGayStraightUp_token_anthro_class_word.ckpt and use in your favorite Stable Diffusion repo!


'sha256sum' is not recognized as an internal or external command,
operable program or batch file.


## Upload checkpoint directly to Pixeldrain
Saves you time and money if you have a slow download speed. Slightly risky, but for some, very worth it. If your rented pod has a fast internet connection, it will upload to pixeldrain very quickly.

### It won't show any output while it is uploading. But if it worked, it will print out a link at at the bottom that you can use to download/share your model.

Take extra care, this method can go wrong in any number of ways.

In [10]:
!wget https://github.com/ManuelReschke/go-pd/releases/download/v1.2.1/go-pd_1.2.1_linux_amd64.tar.gz -O ./go-pd.tar.gz
!tar -xf ./go-pd.tar.gz go-pd
!./go-pd upload "/workspace/Dreambooth-Stable-Diffusion/trained_models/{file_name}"

'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open './go-pd.tar.gz'
'.' is not recognized as an internal or external command,
operable program or batch file.
